In [ ]:
import requests
import json
import time
import datetime as dt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.preprocessing import FunctionTransformer
from sklearn.neural_network import MLPClassifier
from sklearn.neural_network import MLPRegressor
from sklearn import set_config
from sklearn.metrics import mean_absolute_error, mean_squared_error
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
set_config(display='diagram') # Để trực quan hóa pipeline

In [ ]:
# Hàm thu thập dữ liệu từ một khoảng thời gian cụ thể
def collect_data(api_url):
    weather_data = []
    success = False
    while not success:
        print(f"GET: {api_url}")
        r = requests.get(api_url)
        if r.ok:
            # lấy kết quả json
            pydata = json.loads(r.text)
            weather_data.extend(pydata["days"])  # Lưu dữ liệu từ 'days'
            print("Successful! Remaining cost:", pydata.get('remainingCost', 'Unknown'))
            print()
            success = True
        else:
            print("Fail! Try again. Status code:", r.status_code)
            time.sleep(5)
        
    return weather_data


In [ ]:
# Hàm tạo URL API từ các tham số đầu vào
def create_api_url(location, start_date, end_date, api_key, unit_group='metric', include='days'):
    return f'https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/{location}/{start_date}/{end_date}?unitGroup={unit_group}&include={include}&key={api_key}&contentType=json'

# Hàm thu thập dữ liệu trong khoảng thời gian từ nhiều năm
def collect_all_data(start_date, end_date, location, api_key):
    # Tạo URL API
    api_url = create_api_url(location, start_date, end_date, api_key)
    # Thu thập dữ liệu thời tiết
    weather_data = collect_data(api_url)
    
    return weather_data

In [ ]:
# API key (có thể thay đổi)
api_key1 = 'TE7HPVTQ4Q4B7LWSLDSU38BRF'
api_key2 = "VNTCFNZ5VLGZC3FW4F85U8425"
api_key3 = "VWAK37JBEPXKWXTP7U8MQG9FR"

# Thông số đầu vào (có thể thay đổi)
location = 'hanoi'
start_date = '2022-09-01'
end_date = '2024-09-01'

In [ ]:
# # Thu thập dữ liệu
# all_weather_data = collect_all_data(start_date, end_date, location, api_key3)

# # In số lượng dữ liệu thu thập được
# print('Number of data:', len(all_weather_data))

# # Lưu dữ liệu vào DataFrame và xuất ra CSV
# data_df = pd.DataFrame(all_weather_data)
# data_df.rename(lambda name: name[0].upper() + name[1:], axis='columns', inplace=True)
# data_df.to_csv('historical_weather_data.csv', index=False)

# # Hiển thị 5 dòng đầu tiên của dữ liệu
# data_df.head()

In [ ]:
# Đọc dữ liệu từ file CSV
csv_file_path = 'historical_weather_data.csv'  # Đường dẫn đến file CSV
all_weather_data = pd.read_csv(csv_file_path)

# In số lượng dữ liệu đã tải về
print('Number of data loaded from CSV:', len(all_weather_data))


In [ ]:
# Kiểm tra kiểu dữ liệu của các cột
print(all_weather_data.dtypes)

In [ ]:
# Kiểm tra số lượng giá trị không bị thiếu trong mỗi cột
non_null_counts = all_weather_data.count()

# Tính tỷ lệ phần trăm giá trị thiếu trong mỗi cột
missing_percentage = all_weather_data.isnull().mean() * 100

# Tạo DataFrame từ non-null counts và missing percentage
summary_df = pd.DataFrame({
    'Column Name': all_weather_data.columns,
    'Non-null Count': all_weather_data.count().values,
    'Missing Percentage': all_weather_data.isnull().mean().values * 100
})

# In bảng
print(summary_df)

# Lọc ra các cột có tỷ lệ dữ liệu thiếu lớn hơn 20%
missing_columns = missing_percentage[missing_percentage > 20]
print("\nColumns with more than 20% missing data:")
print(missing_columns)

# Tìm các cột có ít hơn 50 giá trị không bị thiếu
low_data_columns = non_null_counts[non_null_counts < 50]
print("\nColumns with less than 50 non-null values:")
print(low_data_columns)

In [ ]:
# In ra thông tin dữ liệu số cột trước khi loại bỏ
print("Columns before dropping 'Preciptype':")
print(all_weather_data.columns)

# In số lượng cột trước khi loại bỏ
print("Number of columns before dropping 'Preciptype':", len(all_weather_data.columns))

# Loại bỏ cột 'Preciptype' khỏi DataFrame
all_weather_data = all_weather_data.drop(columns=['Preciptype'])

# In ra thông tin dữ liệu sau khi loại bỏ cột
print("Columns after dropping 'Preciptype':")
print(all_weather_data.columns)

# In số lượng cột sau khi loại bỏ
print("Number of columns after dropping 'Preciptype':", len(all_weather_data.columns))

In [ ]:
# Đường dẫn đến file CSV mới
new_csv_file_path = 'cleaned_historical_weather_data.csv'  

# Lưu DataFrame vào file CSV
all_weather_data.to_csv(new_csv_file_path, index=False)

# Xác nhận rằng dữ liệu đã được lưu
print(f"Data has been saved to {new_csv_file_path}.")


In [ ]:
# Nhập các thư viện cần thiết
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# 1. Tiền xử lý dữ liệu
# Chuyển đổi cột 'Datetime' thành định dạng datetime
all_weather_data['Datetime'] = pd.to_datetime(all_weather_data['Datetime'])

# Chọn các cột đặc trưng và nhãn
X = all_weather_data[['Tempmax', 'Tempmin', 'Feelslikemax', 'Feelslikemin', 
                      'Dew', 'Pressure', 'Cloudcover', 'Visibility', 'Solarradiation']]
y_temp = all_weather_data['Temp']  # Nhãn cho dự đoán nhiệt độ
y_humidity = all_weather_data['Humidity']  # Nhãn cho dự đoán độ ẩm

# Tách dữ liệu thành tập huấn luyện và kiểm tra (80-20)
X_train, X_test, y_train_temp, y_test_temp = train_test_split(X, y_temp, test_size=0.2, random_state=42)
_, _, y_train_humidity, y_test_humidity = train_test_split(X, y_humidity, test_size=0.2, random_state=42)

# 2. Xây dựng mô hình hồi quy tuyến tính
linear_reg_temp = LinearRegression()
linear_reg_humidity = LinearRegression()

# 3. Huấn luyện mô hình
linear_reg_temp.fit(X_train, y_train_temp)
linear_reg_humidity.fit(X_train, y_train_humidity)

# 4. Dự đoán
# Dự đoán nhiệt độ
y_pred_temp = linear_reg_temp.predict(X_test)
# Dự đoán độ ẩm
y_pred_humidity = linear_reg_humidity.predict(X_test)

In [ ]:
# Chuyển đổi chỉ số của DataFrame thành cột 'Datetime' để sử dụng cho trục x
y_test_temp = pd.Series(y_test_temp, index=X_test.index)
y_pred_temp = pd.Series(y_pred_temp, index=X_test.index)

# Tạo biểu đồ nhiệt độ
plt.figure(figsize=(14, 6))
sns.lineplot(data=y_test_temp, label='Giá trị thực tế', color='blue')
sns.lineplot(data=y_pred_temp, label='Giá trị dự đoán', color='orange', linestyle='--')

# Định dạng trục x
plt.xticks(rotation=45)
plt.title('Nhiệt độ Thực tế và Dự đoán theo Ngày')
plt.xlabel('Ngày')
plt.ylabel('Nhiệt độ (°C)')
plt.legend()
plt.grid()
plt.tight_layout()
plt.show()


In [ ]:
# Chuyển đổi chỉ số của DataFrame thành cột 'Datetime' cho độ ẩm
y_test_humidity = pd.Series(y_test_humidity, index=X_test.index)
y_pred_humidity = pd.Series(y_pred_humidity, index=X_test.index)

# Tạo biểu đồ độ ẩm
plt.figure(figsize=(14, 6))
sns.lineplot(data=y_test_humidity, label='Giá trị thực tế', color='green')
sns.lineplot(data=y_pred_humidity, label='Giá trị dự đoán', color='red', linestyle='--')

# Định dạng trục x
plt.xticks(rotation=45)
plt.title('Độ ẩm Thực tế và Dự đoán theo Ngày')
plt.xlabel('Ngày')
plt.ylabel('Độ ẩm (%)')
plt.legend()
plt.grid()
plt.tight_layout()
plt.show()


In [ ]:
# Đánh giá mô hình cho nhiệt độ
print("Temperature Prediction Metrics:")
print("Mean Absolute Error:", mean_absolute_error(y_test_temp, y_pred_temp))
print("Mean Squared Error:", mean_squared_error(y_test_temp, y_pred_temp))
print("R-squared:", r2_score(y_test_temp, y_pred_temp))

# Đánh giá mô hình cho độ ẩm
print("\nHumidity Prediction Metrics:")
print("Mean Absolute Error:", mean_absolute_error(y_test_humidity, y_pred_humidity))
print("Mean Squared Error:", mean_squared_error(y_test_humidity, y_pred_humidity))
print("R-squared:", r2_score(y_test_humidity, y_pred_humidity))


In [ ]:
from datetime import timedelta
import numpy as np

# 1. Tạo danh sách ngày mới cho khoảng 2 tháng
last_date = all_weather_data['Datetime'].max()
num_days_to_predict = 60  # Khoảng 2 tháng
dates_to_predict = [last_date + timedelta(days=i) for i in range(1, num_days_to_predict + 1)]

# 2. Chuẩn bị các đặc trưng (sử dụng giá trị trung bình)
features_to_predict = X.mean().to_frame().T
features_to_predict = pd.concat([features_to_predict] * num_days_to_predict, ignore_index=True)
features_to_predict.index = dates_to_predict

# 3. Dự đoán nhiệt độ và độ ẩm cho các ngày mới
predicted_temps = linear_reg_temp.predict(features_to_predict)
predicted_humidity = linear_reg_humidity.predict(features_to_predict)

# 4. Tạo DataFrame chứa đầy đủ các cột với dữ liệu dự đoán
predicted_data = pd.DataFrame({
    'Datetime': dates_to_predict,
    'Temp': predicted_temps,
    'Humidity': predicted_humidity
})

# Thêm các cột còn lại theo thứ tự và điền NaN cho các cột không có dữ liệu dự đoán
for col in all_weather_data.columns:
    if col not in predicted_data.columns:
        predicted_data[col] = np.nan

# Sắp xếp lại cột theo thứ tự ban đầu
predicted_data = predicted_data[all_weather_data.columns]

# 5. Ghi thêm dữ liệu dự đoán vào file CSV
predicted_data.to_csv(new_csv_file_path, mode='a', header=False, index=False)

print(f"Predicted data for 2 months has been added to {new_csv_file_path}.")
